In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS231n/assignments/assignment2/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/CS231n/assignments/assignment2/cs231n/datasets
/content/drive/My Drive/CS231n/assignments/assignment2


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('using device:', device)

using device: cuda


In [ ]:
NUM_TRAIN = 49000

transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        self.conv1=nn.Conv2d(in_channel,channel_1,5,stride=1,padding=2)
        nn.init.kaiming_normal_(self.conv1.weight)
        self.conv2=nn.Conv2d(channel_1,channel_2,3,stride=1,padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.fc=nn.Linear(32*32*channel_2,num_classes)
        nn.init.kaiming_normal_(self.fc.weight)
      

    def forward(self, x):
        scores = None
        out=self.conv1(x)
        out=F.relu(out)
        out=self.conv2(out)
        out=F.relu(out)
        out=flatten(out)
        scores=self.fc(out)

        return scores


torch.Size([64, 10])


In [ ]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
def train_part34(model, optimizer, epochs=1):
    model = model.to(device=device)  
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device, dtype=dtype)  
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

        
            optimizer.zero_grad()

         
            loss.backward()

        
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [ ]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = ThreeLayerConvNet(3,channel_1,channel_2,10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
train_part34(model, optimizer)

Iteration 0, loss = 4.7149
Checking accuracy on validation set
Got 105 / 1000 correct (10.50)

Iteration 100, loss = 1.7752
Checking accuracy on validation set
Got 337 / 1000 correct (33.70)

Iteration 200, loss = 1.8295
Checking accuracy on validation set
Got 394 / 1000 correct (39.40)

Iteration 300, loss = 1.6847
Checking accuracy on validation set
Got 420 / 1000 correct (42.00)

Iteration 400, loss = 1.5943
Checking accuracy on validation set
Got 444 / 1000 correct (44.40)

Iteration 500, loss = 1.5398
Checking accuracy on validation set
Got 460 / 1000 correct (46.00)

Iteration 600, loss = 1.4807
Checking accuracy on validation set
Got 449 / 1000 correct (44.90)

Iteration 700, loss = 1.5614
Checking accuracy on validation set
Got 476 / 1000 correct (47.60)



In [ ]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = nn.Sequential(
            nn.Conv2d(3,channel_1,5,stride=1,padding=2),
            nn.ReLU(),
            nn.Conv2d(channel_1,channel_2,3,stride=1,padding=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(32*32*channel_2,10),

)
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3128
Checking accuracy on validation set
Got 138 / 1000 correct (13.80)

Iteration 100, loss = 1.4471
Checking accuracy on validation set
Got 433 / 1000 correct (43.30)

Iteration 200, loss = 1.4417
Checking accuracy on validation set
Got 486 / 1000 correct (48.60)

Iteration 300, loss = 1.2711
Checking accuracy on validation set
Got 495 / 1000 correct (49.50)

Iteration 400, loss = 1.2399
Checking accuracy on validation set
Got 545 / 1000 correct (54.50)

Iteration 500, loss = 1.1611
Checking accuracy on validation set
Got 559 / 1000 correct (55.90)

Iteration 600, loss = 1.1395
Checking accuracy on validation set
Got 557 / 1000 correct (55.70)

Iteration 700, loss = 0.8636
Checking accuracy on validation set
Got 575 / 1000 correct (57.50)

